<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/AutoEncoder_samplecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
import torch.optim as opt
import torchvision
import torchvision.transforms as transforms

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device : {device}")

device : cpu


In [17]:
trainset = torchvision.datasets.MNIST(root='data',
                                      train=True,
                                      transform=transforms.ToTensor(),
                                      download=True)

testset = torchvision.datasets.MNIST(root='data',
                                     train=False,
                                     transform=transforms.ToTensor(),
                                     download=True)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)


In [18]:
# Encoder : Dense(784 -> 128), Dense(128 -> 64), Dense(64 -> 32)
# Decoder : Dense(32 -> 64), Dense(64 -> 128), Dense(128 -> 784)

class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 784)
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded, encoded


In [19]:
model = AE()
print(model)

AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=784, bias=True)
  )
)


In [20]:
loss_fn = nn.MSELoss()
optim = opt.Adam(model.parameters(), lr=0.001)

# Training
epochs = 10

for epoch in range(epochs):
    for data in trainloader:
        inputs, _ = data
        inputs = inputs.view(inputs.size(0), -1)

        optim.zero_grad()

        outputs, _ = model(inputs)
        loss = loss_fn(outputs, inputs)

        loss.backward()
        optim.step()

    print(f"epoch : {epoch}, loss : {loss}")
print("finish")

epoch : 0, loss : 0.022313136607408524
epoch : 1, loss : 0.016836239024996758
epoch : 2, loss : 0.017968159168958664
epoch : 3, loss : 0.01568671315908432
epoch : 4, loss : 0.015186531469225883
epoch : 5, loss : 0.014794282615184784
epoch : 6, loss : 0.014669000171124935
epoch : 7, loss : 0.01414809376001358
epoch : 8, loss : 0.01453017070889473
epoch : 9, loss : 0.013573884032666683
finish


In [22]:
import matplotlib.pyplot as plt
import numpy as np

images, labels = next(iter(testloader))

images_flat = images.view(images.size(0), -1)

recon_images_flat, encoded = model(images_flat)
recon_images = recon_images_flat.view(images.size())

n_imgs = 10
plt.figure(figsize=(20,4))
for i in range(n_imgs):
  ax = plt.subplot(2, n_imgs, i+1)
  plt.imshow(np.transpose(images[i].numpy()))

In [23]:
print(images.size())
print(recon_images_flat.size())
print(recon_images.size())

torch.Size([64, 1, 28, 28])
torch.Size([64, 784])
torch.Size([64, 1, 28, 28])
